In [1]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:33301")
client

<Client: 'tcp://127.0.0.1:33301' processes=4 threads=4, memory=62.82 GiB>

In [2]:
import numpy as np
import xarray as xr
import os.path as op

In [3]:
ddir = '/bettik/uchidat/qg-ensemble/runs/seas-forcing/10percent/'

In [6]:
U = .1
T = 5e5      # seconds
tau0 = .0001
dt = 5e-2
Rd = 5e4     # Rossby radius (50km)
Re4 = 4e3
# f0 = 8e-5
beta = .5
N = 256*4    # Number of horizontal grids
L0 = 80
H = 5e3
dx = L0/N    # Non-dimensionalized horizontal resolution ((L0*Rd)/N)/Rd
diny = 360
Rom = .025
Ekb = .002*2
ntime = 187

percent = 10
nmemb = 100

In [4]:
ds = xr.open_dataset(op.join(ddir, "ctrl/vars.nc"))
    
dsave = ds.chunk({'t':8,'z':1,'y':128,'x':128})
dsave.to_zarr(op.join(ddir, "ctrl/vars.zarr"), mode='w')
dsave

<xarray.Dataset>
Dimensions:  (y: 1024, x: 1024, z: 3, t: 483)
Coordinates:
  * y        (y) float32 0.0 1.0 2.0 3.0 ... 1.021e+03 1.022e+03 1.023e+03
  * x        (x) float32 0.0 1.0 2.0 3.0 ... 1.021e+03 1.022e+03 1.023e+03
  * z        (z) float32 0.0 1.0 2.0
  * t        (t) float32 0.0 1.0 2.0 3.0 4.0 ... 478.0 479.0 480.0 481.0 482.0
Data variables:
    p        (t, z, y, x) float32 dask.array<chunksize=(8, 1, 128, 128), meta=np.ndarray>
    q        (t, z, y, x) float32 dask.array<chunksize=(8, 1, 128, 128), meta=np.ndarray>
    ptr0     (t, z, y, x) float32 dask.array<chunksize=(8, 1, 128, 128), meta=np.ndarray>
    ptr1     (t, z, y, x) float32 dask.array<chunksize=(8, 1, 128, 128), meta=np.ndarray>
    ptr2     (t, z, y, x) float32 dask.array<chunksize=(8, 1, 128, 128), meta=np.ndarray>
    ptr3     (t, z, y, x) float32 dask.array<chunksize=(8, 1, 128, 128), meta=np.ndarray>

In [ ]:
tts = 42
for tt in range(tts,100):
    ds = xr.open_dataset(op.join(ddir, "run%03d/vars.nc" % tt))
    
    dsave = ds.chunk({'t':8,'z':1,'y':128,'x':128})
    dsave.to_zarr(op.join(ddir, "run%03d/vars.zarr" % tt), 
                  mode='w')
    
    ds.close()
    dsave.close()
    print(tt)

42


In [9]:
ds = xr.open_dataset(op.join(ddir, "ctrl/vars.nc")).isel(t=0).chunk({'z':1,'y':128,'x':128})
ds = ds.rename({'z':'Z','y':'YC','x':'XC'})
# ds.coords['timeC'] = ('timeC',.5*(time[1:] + time[:-1]))
ds.coords['Z'] = ('Z',np.array([.03,.13,.6]))
ds.coords['YG'] = ('YG',np.arange(0,L0,dx))
ds.coords['XG'] = ('XG',np.arange(0,L0,dx))
ds.coords['YC'] = ('YC',np.arange(dx/2,L0+dx/2,dx))
ds.coords['XC'] = ('XC',np.arange(dx/2,L0+dx/2,dx))
ds.coords['Zp1'] = ('Zp1',np.array([0.,.06,.2,1.]))
ds.coords['drF'] = ('Z',np.array([.06,.14,.8]))
ds.coords['drC'] = ('Zp1',np.array([.03,.1,.47,.4]))

for i in range(100):
    dss = xr.open_dataset(op.join(ddir, "run%03d/vars.nc" % i)
                         ).isel(t=0).chunk({'z':1,'y':128,'x':128}
                                          )
    dss = dss.rename({'z':'Z','y':'YC','x':'XC'})
    dss.coords['Z'] = ('Z',np.array([.03,.13,.6]))
    dss.coords['YG'] = ('YG',np.arange(0,L0,dx))
    dss.coords['XG'] = ('XG',np.arange(0,L0,dx))
    dss.coords['YC'] = ('YC',np.arange(dx/2,L0+dx/2,dx))
    dss.coords['XC'] = ('XC',np.arange(dx/2,L0+dx/2,dx))
    dss.coords['Zp1'] = ('Zp1',np.array([0.,.06,.2,1.]))
    dss.coords['drF'] = ('Z',np.array([.06,.14,.8]))
    dss.coords['drC'] = ('Zp1',np.array([.03,.1,.47,.4]))

    ds = xr.concat([ds, dss], 'nmemb')
    dss.close()
    print(i)

ds.coords['nmemb'] = ('nmemb',range(101))
ds.chunk({'Z':1,'Zp1':1,
          'YC':128,'XC':128,'YG':128,'XG':128}
        ).to_zarr(op.join(ddir, "vars_%010d.zarr" % 0), mode='w')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [12]:
ds = xr.open_zarr(op.join(ddir, "vars_%010d.zarr" % 0)
                 ).isel(Z=0,Zp1=slice(None,2)
                       ).chunk({'nmemb':9})
ds

<xarray.Dataset>
Dimensions:  (XC: 1024, XG: 1024, YC: 1024, YG: 1024, Zp1: 2, nmemb: 101)
Coordinates:
  * XC       (XC) float64 0.03906 0.1172 0.1953 0.2734 ... 79.8 79.88 79.96
  * XG       (XG) float64 0.0 0.07812 0.1562 0.2344 ... 79.69 79.77 79.84 79.92
  * YC       (YC) float64 0.03906 0.1172 0.1953 0.2734 ... 79.8 79.88 79.96
  * YG       (YG) float64 0.0 0.07812 0.1562 0.2344 ... 79.69 79.77 79.84 79.92
    Z        float64 0.03
  * Zp1      (Zp1) float64 0.0 0.06
    drC      (Zp1) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    drF      float64 dask.array<chunksize=(), meta=np.ndarray>
  * nmemb    (nmemb) int64 0 1 2 3 4 5 6 7 8 9 ... 92 93 94 95 96 97 98 99 100
    t        float32 ...
Data variables:
    p        (nmemb, YC, XC) float32 dask.array<chunksize=(9, 128, 128), meta=np.ndarray>
    ptr0     (nmemb, YC, XC) float32 dask.array<chunksize=(9, 128, 128), meta=np.ndarray>
    ptr1     (nmemb, YC, XC) float32 dask.array<chunksize=(9, 128, 128), meta=np.ndarray>
    ptr2     (nmemb, YC, XC) float32 dask.array<chunksize=(9, 128, 128), meta=np.ndarray>
    ptr3     (nmemb, YC, XC) float32 dask.array<chunksize=(9, 128, 128), meta=np.ndarray>
    q        (nmemb, YC, XC) float32 dask.array<chunksize=(9, 128, 128), meta=np.ndarray>

In [13]:
ds.mean('nmemb',skipna=True).to_zarr(op.join(ddir, "Mean-Z0_%010d.zarr" % 0), mode='w')
ds.isel(nmemb=slice(None,12)).mean('nmemb',skipna=True
                                  ).to_zarr(op.join(ddir, 
                                                    "Mean12-Z0_%010d.zarr" % 0), mode='w')
ds.isel(nmemb=slice(None,25)).mean('nmemb',skipna=True
                                  ).to_zarr(op.join(ddir, 
                                                    "Mean25-Z0_%010d.zarr" % 0), mode='w')
ds.isel(nmemb=slice(None,50)).mean('nmemb',skipna=True
                                  ).to_zarr(op.join(ddir, 
                                                    "Mean50-Z0_%010d.zarr" % 0), mode='w')

In [15]:
dsb = xr.open_zarr(op.join(ddir, "Mean-Z0_%010d.zarr" % 0))
dse = (ds.p - dsb.p).to_dataset(name='p')
dse['q'] = ds.q - dsb.q
dse['ptr0'] = ds.ptr0 - dsb.ptr0
dse['ptr1'] = ds.ptr1 - dsb.ptr1
dse['ptr2'] = ds.ptr2 - dsb.ptr2
dse['ptr3'] = ds.ptr3 - dsb.ptr3
dse.to_zarr(op.join(ddir, "Eddy-Z0_%010d.zarr" % 0), mode='w')

In [18]:
nmembs = 50
dsb = xr.open_zarr(op.join(ddir, "Mean%2d-Z0_%010d.zarr" % (nmembs,0)))
dse = (ds.p.isel(nmemb=slice(None,nmembs)) - dsb.p).to_dataset(name='p')
dse['q'] = ds.q.isel(nmemb=slice(None,nmembs)) - dsb.q
dse['ptr0'] = ds.ptr0.isel(nmemb=slice(None,nmembs)) - dsb.ptr0
dse['ptr1'] = ds.ptr1.isel(nmemb=slice(None,nmembs)) - dsb.ptr1
dse['ptr2'] = ds.ptr2.isel(nmemb=slice(None,nmembs)) - dsb.ptr2
dse['ptr3'] = ds.ptr3.isel(nmemb=slice(None,nmembs)) - dsb.ptr3
dse.to_zarr(op.join(ddir, "Eddy%2d-Z0_%010d.zarr" % (nmembs,0)), mode='w')